In [1]:
import numpy as np

import requests
from bs4 import BeautifulSoup
import pickle

from pymongo import MongoClient
import time

import time
import re

import os

In [2]:
os.chdir('Data')

In [3]:
def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [4]:
def get_job_info(soup):
    
    try:
        job_title = soup.find(name='h3', attrs={'class':'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).text
    except:
        job_title = ''
        
    try:
        company_name = soup.find(name='div', attrs={'class':'icl-u-lg-mr--sm icl-u-xs-mr--xs'}).text
    except:
        company_name = ''
        
    try:
        salary = soup.find(name='span', attrs={'class':'icl-u-xs-mr--xs'}).text
    except:
        salary = ''
    
    try:
        job_description_full = soup.find(name='div', attrs={'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
        
        try:
            job_description_full = soup.find(name='div', attrs={'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
            listed_items = []
            for list_item in job_description_full.find_all(name='li'):
                listed_items.append(list_item.text)
            # Adds '. ' because most list items are missing periods
            listed_items = '. '.join(listed_items)
            listed_items = re.sub('\n', ' ', listed_items)
            # Remove double space
            listed_items = re.sub('  ', ' ', listed_items)
            # Remove double period if list items had periods
            listed_items = re.sub('\.\.', '.', listed_items)
        except:
            listed_items = ''
            
        # Paragraph Items
            
        try:
            paragraph_items = []
            for paragraph_item in job_description_full.find_all(name='p'):
                paragraph_items.append(paragraph_item.text)

            # removes things like: 'Specifically, the Lisp engineer will:'
            paragraph_items = [paragraph_item for paragraph_item in paragraph_items if len(paragraph_item) > 50]
            # Puts all paragraph items into a single string
            paragraph_items = ''.join(paragraph_items)
            paragraph_items = re.sub('\n', ' ', paragraph_items)
            
            if len(paragraph_items) < 1:
                try:
                    job_description_full = soup.find(name='div', attrs={'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
                    paragraph_items = []
                    for paragraph_item in job_description_full.find_all(name='div'):
                        paragraph_items.append(paragraph_item.text)

                    # removes things like: 'Specifically, the Lisp engineer will:'
                    paragraph_items = [paragraph_item for paragraph_item in paragraph_items if len(paragraph_item) > 50]
                    # Puts all paragraph items into a single string
                    paragraph_items = ''.join(paragraph_items)
                    paragraph_items = re.sub('\n', ' ', paragraph_items)
                except: 
                    paragraph_items = ''
   
        except:
            paragraph_items = ''
    
    except:
        listed_items, paragraph_items = ['','']
            
    return job_title, company_name, salary, listed_items, paragraph_items



### Connecting to MongoDB

In [5]:
client = MongoClient()

In [6]:
db = client.jobs_data

In [7]:
db.list_collection_names()

['Seattle_Jobs_Info']

### Importing and removing duplicate URLs

In [8]:
pickle_in = open('individual_listings__Seattle','rb')
seattle_listings_urls = pickle.load(pickle_in)
pickle_in.close()

In [9]:
seattle_listings_urls = [item for sublist in seattle_listings_urls for item in sublist]
seattle_listings_urls = list(set(seattle_listings_urls))

### Looping through and creating individual entries for jobs

In [10]:
# i = 0
# i = 1120
# i = 2190
i = 2580

for url in seattle_listings_urls[2580:]:
    
    new_item = {}
    
    soup = get_soup(url)
    job_title, company_name, salary, listed_items, paragraph_items = get_job_info(soup)
    time.sleep(np.random.poisson(100)/50)
    
    new_item['url'] = url
    new_item['job_title'] = job_title 
    new_item['company_name'] = company_name
    new_item['salary'] = salary
    new_item['listed_items'] = listed_items
    new_item['paragraph_items'] = paragraph_items
    
    time.sleep(np.random.poisson(100)/50)
    
    db.Seattle_Jobs_Info.insert_one(new_item)
    
    if i%10 == 0:
        print(i)
    
    i += 1


2580
2590
2600
2610
2620
2630
2640
2650
2660
2670
2680
2690
2700
2710
2720
2730
2740
2750
2760
2770
2780
2790
2800
2810
2820
2830
2840
2850
2860
2870
2880
2890
2900
2910
2920
2930
2940
2950
2960
2970
2980
2990
3000
3010
3020
3030
3040
3050
3060
3070
3080
3090
3100
3110
3120
3130
3140
3150
3160
3170
3180
3190
3200
3210
3220
3230
3240
3250
3260
3270
3280
3290
3300
3310
3320
3330
3340
3350
3360
3370
3380
3390
3400
3410
3420
3430
